In [1]:
import math
import sys
import importlib

import numpy as np
import pandas as pd
from IPython import display

sys.path.append('C:/Users/Ang/Developer/Projects/NikonTiControl/python')
import api

In [2]:
api.list_property("/Hamamatsu")

['/Hamamatsu/BINNING', '/Hamamatsu/BIT PER CHANNEL', '/Hamamatsu/BUFFER FRAMEBYTES', '/Hamamatsu/BUFFER PIXEL TYPE', '/Hamamatsu/BUFFER ROWBYTES', '/Hamamatsu/BUFFER TOP OFFSET BYTES', '/Hamamatsu/COLORTYPE', '/Hamamatsu/CONTRAST GAIN', '/Hamamatsu/CONTRAST OFFSET', '/Hamamatsu/EXPOSURE TIME', '/Hamamatsu/EXPOSURE TIME CONTROL', '/Hamamatsu/FRAME STAMP PRODUCER', '/Hamamatsu/HIGH DYNAMIC RANGE MODE', '/Hamamatsu/IMAGE DETECTOR PIXEL HEIGHT', '/Hamamatsu/IMAGE DETECTOR PIXEL WIDTH', '/Hamamatsu/IMAGE FRAMEBYTES', '/Hamamatsu/IMAGE HEIGHT', '/Hamamatsu/IMAGE PIXEL TYPE', '/Hamamatsu/IMAGE ROWBYTES', '/Hamamatsu/IMAGE TOP OFFSET BYTES', '/Hamamatsu/IMAGE WIDTH', '/Hamamatsu/INTERNAL FRAME INTERVAL', '/Hamamatsu/INTERNAL FRAME RATE', '/Hamamatsu/LIGHT MODE', '/Hamamatsu/NUMBER OF OUTPUT TRIGGER CONNECTOR', '/Hamamatsu/OUTPUT TRIGGER ACTIVE', '/Hamamatsu/OUTPUT TRIGGER DELAY', '/Hamamatsu/OUTPUT TRIGGER PERIOD', '/Hamamatsu/OUTPUT TRIGGER POLARITY', '/Hamamatsu/READOUT SPEED', '/Hamamatsu/R

In [3]:
api.get_property("/Hamamatsu/BINNING")

'1x1'

In [4]:
api.set_property({"/Hamamatsu/BINNING": "2x2"})

In [34]:
importlib.reload(api)

<module 'api' from 'c:\\Users\\Ang\\Developer\\Projects\\NikonTiControl\\examples\\../python\\api.py'>

In [10]:
api.set_experiment_path("D:/Test")

In [11]:
channels = [
    ("BF", 25),
    ("YFP", 50, 40),
]

api.acquire_multi_channel("image1", channels, 0, 0)

In [82]:
channels = [
    ("BF", 25),
    ("YFP", 50, 40),
    ("RFP", 50, 25),
]

api.acquire_multi_channel("image2", channels, 0, 0)

In [83]:
api.list_ndimage()

[{'name': 'image1',
  'channel_info': [{'name': 'BF', 'width': 1344, 'height': 1024},
   {'name': 'YFP', 'width': 1344, 'height': 1024}],
  'n_images': 2,
  'n_z': 1,
  'n_t': 1},
 {'name': 'image2',
  'channel_info': [{'name': 'BF', 'width': 1344, 'height': 1024},
   {'name': 'YFP', 'width': 1344, 'height': 1024},
   {'name': 'RFP', 'width': 1344, 'height': 1024}],
  'n_images': 3,
  'n_z': 1,
  'n_t': 1}]

In [47]:
class Wellplate384():
    def __init__(self, current_well_name):
        self.spacing_x = -4500
        self.spacing_y = -4500
        x, y = api.get_xy_stage_position()
        row, col = self.well_name_to_row_col(current_well_name)
        self.pos_A1 = ((x - (col - 1) * self.spacing_x), (y - (row - 1) * self.spacing_y))
    
    def get_current_well(self):
        x, y = api.get_xy_stage_position()
        col = int(np.round((x - self.pos_A1[0]) / self.spacing_x)) + 1
        row = int(np.round((y - self.pos_A1[1]) / self.spacing_y)) + 1
        return self.row_col_to_well_name(row, col)
    
    def get_well_position(self, well_name):
        row, col = self.well_name_to_row_col(well_name)
        x = self.pos_A1[0] + (col - 1) * self.spacing_x
        y = self.pos_A1[1] + (row - 1) * self.spacing_y
        return (x, y)
        
    def well_name_to_row_col(self, well_name):
        row = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'].index(well_name[0]) + 1
        col = int(well_name[1:])
        return row, col

    def row_col_to_well_name(self, row, col):
        row_name = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'][row - 1]
        col_name = str(col)
        return row_name+col_name

In [50]:
wp = Wellplate384(current_well_name="A1")

In [51]:
wp.pos_A1

(51868.4, 36161.3)

In [55]:
def go_to_position(well, ix, iy):    
    x, y = wp.get_well_position(well)
    api.set_xy_stage_position(x - 250 * ix, y - 250 * iy)

In [59]:
go_to_position("A1", 1, 1)

In [53]:
sites = []

offset = 'A1'
n_col = 3
n_row = 2

n_site_x = 2
n_site_y = 2
site_spacing = -250

remove_wells = []

for row in range(n_row):
    col_list = list(range(n_col))
    if row % 2 != 0:
        col_list.reverse()
    
    row_offset, col_offset = wp.well_name_to_row_col(offset)
    for col in col_list:            
        well_name = wp.row_col_to_well_name(row+row_offset, col+col_offset)
        well_position = wp.get_well_position(well_name)
    
        if well_name in remove_wells:
            continue
        i_site = 0
        for i_site_x in range(n_site_x):
            i_site_y_list = list(range(n_site_y))
            if col % 2 != 0:
                i_site_y_list.reverse()
                
            for i_site_y in i_site_y_list:
                site_position_x = well_position[0] + site_spacing * i_site_x
                site_position_y = well_position[1] + site_spacing * i_site_y
                
                sites.append([well_name, i_site, site_position_x, site_position_y])
                i_site += 1

sites = pd.DataFrame(sites, columns=["well", "site", "x", "y"])

In [54]:
sites

,well,site,x,y
0,A1,0,51868.4,36161.3
1,A1,1,51868.4,35911.3
2,A1,2,51618.4,36161.3
3,A1,3,51618.4,35911.3
4,A2,0,47368.4,35911.3
5,A2,1,47368.4,36161.3
6,A2,2,47118.4,35911.3
7,A2,3,47118.4,36161.3
8,A3,0,42868.4,36161.3
9,A3,1,42868.4,35911.3


In [74]:
def search_focus_range(z_range, z_step_size, z_center, z_max_safety=None):
    try:
        api.enable_pfs()
    except RuntimeError:
        n_steps = math.ceil(z_range / z_step_size)
        z_min = z_center - (n_steps / 2) * z_step_size
        z_max = z_center + (n_steps / 2) * z_step_size
        if z_max_safety and (z_max > z_max_safety):
            z_max = z_max_safety
        if z_max_safety and (z_min > z_max_safety):
            raise ValueError("z_min= {}, higher than z_max_safety ={}".format(z_min, z_max_safety))
        n_steps = math.ceil((z_max - z_min) / z_step_size)
        z_list = list(np.linspace(z_min, z_max, n_steps))
        z_list.reverse()
        for z_pos in z_list:
            print("set z={}".format(z_pos))
            api.set_z_stage_position(z_pos)
            api.wait_z_stage()
            try:
                api.enable_pfs()
                return
            except:
                continue
        raise RuntimeError("Focus search failed")

def find_focus():
    try:
        api.enable_pfs()
        return
    except RuntimeError:
        pass

    z_initial = api.get_z_stage_position()
    z_max_safety = None

    try:
        print("search focus within 30 um:")
        search_focus_range(30, 5, z_initial, z_max_safety)
        print("focus found")
        return
    except RuntimeError:
        pass

    try:
        print("search focus within 75 um:")
        search_focus_range(75, 5, z_initial, z_max_safety)
        print("focus found")
        return
    except RuntimeError:
        pass

    try:
        print("search focus within 150 um:")
        search_focus_range(150, 5, z_initial, z_max_safety)
        print("focus found")
    except RuntimeError:
        print("failed: go back to z={}".format(z_initial))
        api.set_z_stage_position(z_initial)
        api.wait_z_stage()
        return

In [75]:
channels = [
    ("BF", 25),
    ("YFP", 50, 40),
]

def image_sites(site_list, seq_no=0):
    for (i, s) in site_list.iterrows():
        display.display("%d/%d. Well %s. Site %s. Position: (%.1f, %.1f)." % (i, len(sites), s.well, s.site, s.x, s.y))
        api.set_xy_stage_position(s.x, s.y)
        api.wait_xy_stage()
        find_focus()
        
        imname = "%s-%03d" % (s.well, s.site)
        api.acquire_multi_channel(imname, channels, 0, seq_no)

In [77]:
image_sites(sites)

'0/24. Well A1. Site 0. Position: (51868.4, 36161.3).'

search focus within 30 um:
set z=2799.1
set z=2793.1
set z=2787.1
set z=2781.1
set z=2775.1
set z=2769.1
search focus within 75 um:
set z=2821.6
set z=2816.2428571428572
set z=2810.885714285714
set z=2805.5285714285715
set z=2800.1714285714284
set z=2794.8142857142857
set z=2789.4571428571426
set z=2784.1
set z=2778.7428571428572
set z=2773.385714285714
set z=2768.0285714285715
set z=2762.6714285714284
set z=2757.3142857142857
set z=2751.9571428571426
focus found


'1/24. Well A1. Site 1. Position: (51868.4, 35911.3).'

'2/24. Well A1. Site 2. Position: (51618.4, 36161.3).'

'3/24. Well A1. Site 3. Position: (51618.4, 35911.3).'

...

'9/24. Well A3. Site 1. Position: (42868.4, 35911.3).'

'10/24. Well A3. Site 2. Position: (42618.4, 36161.3).'

'11/24. Well A3. Site 3. Position: (42618.4, 35911.3).'

'12/24. Well B3. Site 0. Position: (42868.4, 31661.3).'

KeyboardInterrupt: 